# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [27]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geoviews
# Import API key
from api_keys import geoapify_key


In [28]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bilibino,68.0546,166.4372,246.61,96,99,1.53,RU,1710278593
1,1,luderitz,-26.6481,15.1594,293.59,65,97,8.25,NaN,1710278593
2,2,alaghsas,17.0187,8.0168,302.97,8,59,5.22,NE,1710278594
3,3,adamstown,-25.0660,-130.1015,298.83,76,86,3.06,PN,1710278594
4,4,edinburgh of the seven seas,-37.0676,-12.3116,290.22,70,100,5.83,SH,1710278594


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [29]:
# Configure and customise the map plot

humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500
)

# Display the map plot

hvplot.show(humidity_map)

Launching server at http://localhost:63356


### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [30]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_conditions_df = city_data_df[(city_data_df['Max Temp'] > 294) & 
                                   (city_data_df['Max Temp'] < 300) & 
                                   (city_data_df['Wind Speed'] < 4.5) & 
                                   (city_data_df['Cloudiness'] == 0)].copy()

# Drop any rows with null values
ideal_conditions_df.dropna(inplace=True)

# Display sample data
display(ideal_conditions_df.head())

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
210,210,grand island,40.9250,-98.3420,294.12,21,0,3.60,US,1710278630
244,244,kuchaiburi,22.2667,86.1667,296.54,33,0,0.65,IN,1710278638
245,245,marsaskala,22.2667,86.1667,296.54,33,0,0.65,IN,1710278638
267,267,rajgarh,28.6333,75.3833,294.35,31,0,2.74,IN,1710278641
303,303,balasore,21.4942,86.9317,297.42,91,0,2.56,IN,1710278648


### Step 3: Create a new DataFrame called `hotel_df`.

In [31]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API

hotel_df['Hotel Name'] = ''

# Display sample data

display(hotel_df)

,City,Country,Lat,Lng,Humidity,Hotel Name
0,bilibino,RU,68.0546,166.4372,96,
1,luderitz,NaN,-26.6481,15.1594,65,
2,alaghsas,NE,17.0187,8.0168,8,
3,adamstown,PN,-25.0660,-130.1015,76,
4,edinburgh of the seven seas,SH,-37.0676,-12.3116,70,
...,...,...,...,...,...,...
593,usinsk,RU,65.9939,57.5281,94,
594,arteche,PH,12.2645,125.4048,72,
595,calabozo,VE,8.9242,-67.4293,24,
596,san pedro pochutla,MX,15.7333,-96.4667,53,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [32]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'categories': 'accommodation.hotel',
    #'radius': radius,
    'limit': 20,
    'apiKey': geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    lat = row['Lat']
    lon = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    
    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    if name_address.status_code == 200:
        name_address = name_address.json()
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bilibino - nearest hotel: No hotel found
luderitz - nearest hotel: Seaview Hotel Zum Sperrgebiet
alaghsas - nearest hotel: Hôtel Zagado
adamstown - nearest hotel: No hotel found
edinburgh of the seven seas - nearest hotel: No hotel found
cabo san lucas - nearest hotel: Comfort Rooms
blackmans bay - nearest hotel: Villa Howden
port-aux-francais - nearest hotel: Keravel
georgetown - nearest hotel: Page 63 hostel
punta arenas - nearest hotel: Hotel Lacolet
yellowknife - nearest hotel: No hotel found
waitangi - nearest hotel: Hotel Chathams
pimentel - nearest hotel: Hospedaje Calle del Sol
vilyuchinsk - nearest hotel: Победа
progreso - nearest hotel: Centro Vacacional Obrero CTM
hasaki - nearest hotel: Choshi Plaza Hotel
puerto baquerizo moreno - nearest hotel: Dory's House
mogadishu - nearest hotel: Villa Rays Restaurant & Hotel
kingston - nearest hotel: Aphrodite Hotel
college - nearest hotel: BEST WESTERN PLUS Pioneer Park Inn
muang xay - nearest hotel: ໂຮງແຮມງາມລາ

,City,Country,Lat,Lng,Humidity,Hotel Name
0,bilibino,RU,68.0546,166.4372,96,No hotel found
1,luderitz,NaN,-26.6481,15.1594,65,Seaview Hotel Zum Sperrgebiet
2,alaghsas,NE,17.0187,8.0168,8,Hôtel Zagado
3,adamstown,PN,-25.0660,-130.1015,76,No hotel found
4,edinburgh of the seven seas,SH,-37.0676,-12.3116,70,No hotel found
...,...,...,...,...,...,...
593,usinsk,RU,65.9939,57.5281,94,общежитие Лукойл-Коми
594,arteche,PH,12.2645,125.4048,72,No hotel found
595,calabozo,VE,8.9242,-67.4293,24,Gran Hotel Guarico
596,san pedro pochutla,MX,15.7333,-96.4667,53,Olimpia


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [33]:
# Configure the map plot

#hotel_points = geoviews.Points(hotel_df, ['Lng', 'Lat'], ['Humidity', 'Hotel Name', 'Country'])

#hotel_map = (gts.CartoDark * hotel_points.opts(size=5, color='red', line_color='white', tools=['hover'], width=800, height=600))

# Display the map
hotel_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map plot

hvplot.show(hotel_map)

hotel_map

Launching server at http://localhost:50560


AssertionError: 